## GAN
全称Generative Adversarial Networks,即生成-对抗网络，是一种NN 模型，是建立在复杂分布数据的基础上的无监督学习方。
GAN中有两大模块，第一模块Model是生成模型（Generative Model,G）第二个模块是判别模型（对抗模型，Discriminative Model D）。
GAN的学习便是两个模块的博弈对抗。 在GAN的理论中，并不要求G和D都是神经网络，只要是能拟合相应生成和判别的函数即可。

## GAN的内部定义
数据方面：我们有真实的数据，例如图片，还有自定义的噪声，将噪声数据输入到G中，G对噪声进行编码处理，从低维到高维，最终生成一张图片，最后将噪声与真实的图片给D，D进行判别处理，最后对结果进行优化。
我们需要明确一点**G和D是分开训练的**。两者在训练的过程中分别进步，一开始初始化G和D中的权重，此时G和D只是几层网络而已，下面通过向G输入噪声，G此时生成的数据毫无实际意义，可能是乱码或者其他未知的图片，之后，通过人工操作，将数据输入到D中，我们自己让D对这些数据判别为False，之后输入真实的图片，手动使D判别为True，之后对D和G的参数进行优化，优化的目标是以真实图片训练参数，进行n次训练后，参数偏向稳定状态，此时我们如果输入随机数到G中，G便会生成图片，且D会自行将生成的图片判断为True

## GAN的Torch实现

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import transforms
import torchvision

import os
import sys

sys.path.append('../')
sys.path.append('../d2l.py')
import lmy

/Users/zane/miniforge3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


创建图片所在的文件夹

In [2]:
if not os.path.exists('./img'):
    os.mkdir('./img')

In [3]:
def to_img(x):
    """将tensor转为图片格式"""
    x = (x + 1) * .5
    x = x.clamp(0, 1) # clamp 夹紧，将x的值限制在0-1之间
    x = x.view(-1, 1, 28, 28) #view()函数的作用，将多行的tensor转换为多个二维的图片
    return x

In [4]:
from torchvision.datasets import FashionMNIST
from torch.utils import data

batch_size = 128
num_epochs = 100
z_dimension = 100
# 图像预处理
img_transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为tensor
    transforms.Normalize([0.5], [0.5])  # 归一化
])
# 读取数据
mnist_train = FashionMNIST(root='../lmy/data', train=True, transform=img_transform, download=False)
bags_data = []
for i, (img, label) in enumerate(mnist_train):
    if label == 8:
        bags_data.append(img)
train_iter = data.DataLoader(bags_data, batch_size, shuffle=False, num_workers=4)

定义判别器D网络

In [5]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.discriminator = nn.Sequential(
            nn.Linear(784, 256), nn.LeakyReLU(.2),
            nn.Linear(256, 256), nn.LeakyReLU(.2),
            nn.Linear(256, 1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.discriminator(x)

    def __call__(self, x):
        return self.forward(x)

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.generator = nn.Sequential(
            nn.Linear(100, 256),  # 用线性变换将输入映射到256维 输入是100维
            nn.ReLU(),  # relu激活
            nn.Linear(256, 256),  # 线性变换
            nn.ReLU(),  # relu激活
            nn.Linear(256, 784),  # 线性变换
            nn.Tanh()  # Tanh激活使得生成数据分布在【-1,1】之间，因为输入的真实数据的经过transforms之后也是这个分布
        )

    def forward(self, x):
        return self.generator(x)

    def __call__(self, x):
        return self.forward(x)

实例化网络

In [7]:
D = Discriminator()
G = Generator()
devices = lmy.getGPU(contain_cpu=True)
cuda_available = False
if 'cuda' in devices[0].type:
    cuda_available = True

if cuda_available:
    D = D.to(devices[0])
    G = G.to(devices[0])

评价标准和优化器

In [8]:
criterion = nn.BCELoss() # 二进制交叉熵 因为结果只有True和False
g_optimizer = torch.optim.Adam(G.parameters(),lr=.001)
d_optimizer = torch.optim.Adam(D.parameters(),lr=.001)

In [10]:

for epoch in range(num_epochs):
    for i, train_imgs in enumerate(train_iter):
        num_imgs = len(train_imgs)  # = batch_size
        #====================训练判别器D==================#
        # 判别器的训练分为两个部分：1 真实图像判别为真 2 生成图像判别为假

        train_imgs = train_imgs.reshape(num_imgs, -1)  # 拉平 将一个batch所有的图片放到一个tensor中
        # print(train_imgs.shape) # torch.Size([128, 784])

        real_img = Variable(train_imgs)  #tensor转变为Vairable类型的变量
        real_label = Variable(torch.ones(num_imgs))  # 定义真实的图片label为1
        fake_label = Variable(torch.zeros(num_imgs))  # 定义虚假的图片label为0
        if cuda_available:
            real_img, real_label, fake_label = real_img.to(devices[0]), real_label.to(devices[0]), fake_label.to(
                devices[0])

        # 计算真实图片的损失
        d_real_out = D(real_img).squeeze()  # 真实图片的输出
        d_real_loss = criterion(d_real_out, real_label)
        real_scores = d_real_out

        # 计算假图片的损失
        z = Variable(torch.randn(num_imgs, z_dimension))
        if cuda_available:
            z = z.to(devices[0])

        fake_img = G(z).detach()  # 生成假图片
        fake_out = D(fake_img).squeeze()  # 使用判别器对假图片进行判断
        d_fake_loss = criterion(fake_out, fake_label)
        fake_scores = fake_out
        d_loss = d_real_loss + d_fake_loss  # 损失包括真损失和假损失
        d_optimizer.zero_grad()
        d_loss.backward()
        d_optimizer.step()

        #====================训练生成器G==================#
        # 目的：希望生成的假图片被判别器D判断为True
        # 生成器的训练分两个部分：1 生成假图片 2 判别器对假图片判别为真
        # 过程中 将判别器参数固定，将假的图片传入判别器的结果与真实的label对应

        z = Variable(torch.randn(num_imgs, z_dimension))
        if cuda_available:
            z = z.to(devices[0])
        fake_img = G(z).detach()
        output = D(fake_img).squeeze()
        g_loss = criterion(output, real_label)
        g_optimizer.zero_grad()  # 梯度归0
        g_loss.backward()
        g_optimizer.step()
        # 打印中间的损失
        if (i + 1) % 20 == 0:
            print(
                f'Epoch[{epoch}/{num_epochs}],d_loss:{d_loss.data.item():.6f},g_loss:{g_loss.data.item():.6f} ,D real: {real_scores.data.mean():.6f},D fake: {fake_scores.data.mean():.6f}')  # 打印的是真实图片的损失均值
        if epoch == 0:
            real_images = to_img(real_img.cpu().data)
            torchvision.utils.save_image(real_images, './img/real_images.png')

    fake_images = to_img(fake_img.cpu().data)
    torchvision.utils.save_image(fake_images, f'./img/fake_images-{epoch + 1}.png')

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x11b4858b0>
Traceback (most recent call last):
  File "/Users/zane/miniforge3/envs/torch/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "/Users/zane/miniforge3/envs/torch/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 1316, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Epoch[0/100],d_loss:0.000001,g_loss:14.658181 ,D real: 1.000000,D fake: 0.000001
Epoch[0/100],d_loss:0.000002,g_loss:14.747150 ,D real: 1.000000,D fake: 0.000002
Epoch[1/100],d_loss:0.000001,g_loss:14.753455 ,D real: 1.000000,D fake: 0.000001
Epoch[1/100],d_loss:0.000001,g_loss:14.400969 ,D real: 1.000000,D fake: 0.000001
Epoch[2/100],d_loss:0.000001,g_loss:14.398615 ,D real: 1.000000,D fake: 0.000001
Epoch[2/100],d_loss:0.000001,g_loss:14.841447 ,D real: 1.000000,D fake: 0.000001
Epoch[3/100],d_loss:0.000001,g_loss:14.792832 ,D real: 1.000000,D fake: 0.000001
Epoch[3/100],d_loss:0.000001,g_loss:14.731848 ,D real: 1.000000,D fake: 0.000001
Epoch[4/100],d_loss:0.000001,g_loss:14.868746 ,D real: 1.000000,D fake: 0.000001
Epoch[4/100],d_loss:0.000001,g_loss:14.528069 ,D real: 1.000000,D fake: 0.000001
Epoch[5/100],d_loss:0.000001,g_loss:14.464863 ,D real: 1.000000,D fake: 0.000001
Epoch[5/100],d_loss:0.000001,g_loss:14.584456 ,D real: 1.000000,D fake: 0.000001
Epoch[6/100],d_loss:0.000001